In [1]:
import easyocr
import cv2
from tqdm import tqdm
import os
import csv
import re

reader = easyocr.Reader(['en'], gpu=True)

def extract_text_from_image(image_path):
    image = cv2.imread(image_path)
    results = reader.readtext(image)
    extracted_text = ' '.join([result[1] for result in results])
    return extracted_text

def parse_measurement(text):
    # Expanded pattern to catch more variations
    pattern = r'(\d+(?:\.\d+)?)\s*(gram|centimetre|ounce|kilogram|g|cm|oz|kg|foot|ft|inch|in|metre|m|millimetre|mm|ton|t|volt|v|watt|w|kilovolt|kv|kilowatt|kw|pound|lb|yard|yd|millivolt|mv|microgram|μg)'
    match = re.search(pattern, text.lower())
    if match:
        value, unit = match.groups()
        # Normalize units
        unit_map = {
            'g': 'gram', 'cm': 'centimetre', 'oz': 'ounce', 'kg': 'kilogram',
            'ft': 'foot', 'in': 'inch', 'm': 'metre', 'mm': 'millimetre',
            't': 'ton', 'v': 'volt', 'w': 'watt', 'kv': 'kilovolt',
            'kw': 'kilowatt', 'lb': 'pound', 'yd': 'yard', 'mv': 'millivolt',
            'μg': 'microgram'
        }
        unit = unit_map.get(unit, unit)
        return f"{float(value)} {unit}"
    return ""

image_folder = '../images'
output_file = 'test_out.csv'

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['index', 'prediction'])
    
    for index, file in enumerate(tqdm(os.listdir(image_folder))):
        image_path = os.path.join(image_folder, file)
        try:
            extracted_text = extract_text_from_image(image_path)
            prediction = parse_measurement(extracted_text)
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
            prediction = ""
        writer.writerow([index, prediction])

print(f"CSV file '{output_file}' has been created with the predictions.")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
d:\Python\Lib\site-packages\easyocr\detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to th

CSV file 'test_out.csv' has been created with the predictions.


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

def normalize_prediction(pred):
    if not pred:
        return ""
    parts = pred.split()
    if len(parts) != 2:
        return ""
    try:
        value = float(parts[0])
        unit = parts[1]
        return f"{value:.2f} {unit}"
    except ValueError:
        return ""

# Load the predictions and ground truth
predictions_df = pd.read_csv('test_out.csv')
ground_truth_df = pd.read_csv('../test.csv')  # Adjust the path as needed

# Ensure the dataframes are sorted by index
predictions_df = predictions_df.sort_values('index').reset_index(drop=True)
ground_truth_df = ground_truth_df.sort_values('index').reset_index(drop=True)

# Normalize predictions and ground truth
predictions_df['normalized_prediction'] = predictions_df['prediction'].apply(normalize_prediction)
ground_truth_df['normalized_ground_truth'] = ground_truth_df['measurement'].apply(normalize_prediction)

# Create binary labels (1 for non-empty predictions, 0 for empty)
y_true = (ground_truth_df['normalized_ground_truth'] != "").astype(int)
y_pred = (predictions_df['normalized_prediction'] != "").astype(int)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)

print(f"F1 Score: {f1:.4f}")

# Calculate accuracy for non-empty predictions
non_empty_mask = (y_true == 1) & (y_pred == 1)
correct_predictions = (predictions_df.loc[non_empty_mask, 'normalized_prediction'] == 
                       ground_truth_df.loc[non_empty_mask, 'normalized_ground_truth'])
accuracy = correct_predictions.mean() if len(correct_predictions) > 0 else 0

print(f"Accuracy for non-empty predictions: {accuracy:.4f}")

# Display some example comparisons
print("\nExample comparisons:")
comparison_df = pd.DataFrame({
    'Ground Truth': ground_truth_df['normalized_ground_truth'],
    'Prediction': predictions_df['normalized_prediction']
})
print(comparison_df.head(10))
